In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pytidycensus as tc
import requests

C:\Users\lukem\AppData\Local\Temp\ipykernel_18180\3992969218.py:5: UserWarning: Mapping functions unavailable due to import error: NameError. To use mapping features, ensure all dependencies are properly installed: pip install pytidycensus[map]
  import pytidycensus as tc


In [2]:
redfin_metro = pd.read_csv(r'C:\Users\lukem\OneDrive\Desktop\NSS\Capstone\Capstone-Project\Data\redfin_data.csv',encoding='utf-16',sep='\t')

In [3]:
redfin_metro['Median Sale Price'] = (redfin_metro['Median Sale Price'].str.replace('$','').str.replace('K','000').astype(float))

In [4]:
cols = ['Homes Sold', 'New Listings', 'Inventory']
for col in cols:
    redfin_metro[col] = redfin_metro[col].str.replace(',', '').astype(int)

In [5]:
redfin_metro['Month of Period End'] = pd.to_datetime(redfin_metro['Month of Period End'],format='%B %Y')

In [6]:
redfin_metro['Year'] = redfin_metro['Month of Period End'].dt.year

In [7]:
redfin_metro = redfin_metro[(redfin_metro['Year'] >= 2020) & (redfin_metro['Year'] <= 2023)]

In [8]:
redfin_metro.head()

,Region,Month of Period End,Median Sale Price,Median Sale Price MoM,Median Sale Price YoY,Homes Sold,Homes Sold MoM,Homes Sold YoY,New Listings,New Listings MoM,...,Inventory,Inventory MoM,Inventory YoY,Days on Market,Days on Market MoM,Days on Market YoY,Average Sale To List,Average Sale To List MoM,Average Sale To List YoY,Year
83,"Nashville, TN metro area",2020-01-01,300000.0,-5.7%,2.9%,2195,-25.1%,16.2%,3092,57.4%,...,8572,2.5%,-7.6%,73,9.0,0.0,98.3%,-0.1%,0.3%,2020
84,"Nashville, TN metro area",2020-02-01,315000.0,5.0%,8.6%,2431,10.8%,15.3%,2998,-3.0%,...,8607,0.4%,-8.3%,72,-1.0,-6.0,98.5%,0.2%,0.1%,2020
85,"Nashville, TN metro area",2020-03-01,326000.0,3.4%,8.5%,2953,21.5%,7.5%,3756,25.3%,...,9043,5.1%,-6.0%,63,-9.0,-3.0,99.0%,0.4%,0.5%,2020
86,"Nashville, TN metro area",2020-04-01,327000.0,0.4%,7.7%,2579,-12.7%,-13.0%,3466,-7.7%,...,9535,5.4%,-7.5%,56,-7.0,-1.0,99.1%,0.1%,0.6%,2020
87,"Nashville, TN metro area",2020-05-01,325000.0,-0.6%,4.3%,2820,9.3%,-19.8%,3713,7.1%,...,9838,3.2%,-7.8%,53,-3.0,-3.0,98.9%,-0.2%,0.2%,2020


In [9]:
redfin_metro['weighted_price'] = (redfin_metro['Median Sale Price'] * redfin_metro['Homes Sold'])

In [10]:
annual_prices = (redfin_metro.groupby('Year').agg(total_weighted_price=('weighted_price', 'sum'), total_sales=('Homes Sold', 'sum')).reset_index())

In [11]:
annual_prices['Weighted_Avg_Sale_Price'] = (annual_prices['total_weighted_price'] /annual_prices['total_sales'])

In [12]:
annual_prices

,Year,total_weighted_price,total_sales,Weighted_Avg_Sale_Price
0,2020,1.280479e+10,38234,334905.921431
1,2021,1.484669e+10,37286,398184.036904
2,2022,1.472740e+10,31384,469264.625287
3,2023,1.239891e+10,26570,466650.884456


In [13]:
redfin_zip = pd.read_csv(r'C:\Users\lukem\OneDrive\Desktop\NSS\Capstone\Capstone-Project\Data\zip_code_market_tracker.tsv000',sep='\t',encoding='utf-8',low_memory=False)

In [14]:
redfin_zip['PERIOD_END'] = pd.to_datetime(redfin_zip['PERIOD_END'])
redfin_zip['PERIOD_BEGIN'] = pd.to_datetime(redfin_zip['PERIOD_BEGIN'])

In [15]:
redfin_zip = redfin_zip[redfin_zip['REGION_TYPE'] == 'zip code']

In [16]:
redfin_zip = redfin_zip[redfin_zip['PARENT_METRO_REGION'] == 'Nashville, TN']

In [17]:
redfin_zip = redfin_zip[redfin_zip['PROPERTY_TYPE'] == 'Single Family Residential']

In [18]:
redfin_zip = redfin_zip[(redfin_zip['PERIOD_END'].dt.year >= 2020) &(redfin_zip['PERIOD_END'].dt.year <= 2023)]

In [19]:
redfin_zip['ZIP'] = (redfin_zip['REGION'].str.replace('Zip Code: ', '', regex=False).str.zfill(5))

In [20]:
redfin_zip['Year'] = redfin_zip['PERIOD_END'].dt.year
redfin_zip['Month'] = redfin_zip['PERIOD_END'].dt.month

In [21]:
metro_nashville_zips = ['37201','37203','37204','37205','37206','37207',
                        '37208','37209','37210','37211','37212','37214',
                        '37215','37216','37217','37218','37220','37221',
                        '37127','37128','37129','37130','37027','37064',
                        '37067','37069','37066','37075','37076','37087',
                        '37122']

In [22]:
redfin_zip = redfin_zip[redfin_zip['ZIP'].isin(metro_nashville_zips)]

In [23]:
redfin_zip

,PERIOD_BEGIN,PERIOD_END,PERIOD_DURATION,REGION_TYPE,REGION_TYPE_ID,TABLE_ID,IS_SEASONALLY_ADJUSTED,REGION,CITY,STATE,...,PRICE_DROPS_YOY,OFF_MARKET_IN_TWO_WEEKS,OFF_MARKET_IN_TWO_WEEKS_MOM,OFF_MARKET_IN_TWO_WEEKS_YOY,PARENT_METRO_REGION,PARENT_METRO_REGION_METRO_CODE,LAST_UPDATED,ZIP,Year,Month
1757,2023-08-01,2023-10-31,90,zip code,2,15686,False,Zip Code: 37087,NaN,Tennessee,...,NaN,0.334975,0.026032,-0.081055,"Nashville, TN",34980,2025-12-15 14:55:40.754 Z,37087,2023,10
3043,2020-08-01,2020-10-31,90,zip code,2,15704,False,Zip Code: 37127,NaN,Tennessee,...,NaN,0.765306,0.089868,0.386928,"Nashville, TN",34980,2025-12-15 14:55:40.754 Z,37127,2020,10
23522,2023-07-01,2023-09-30,90,zip code,2,15760,False,Zip Code: 37209,NaN,Tennessee,...,NaN,0.452555,-0.054798,-0.096741,"Nashville, TN",34980,2025-12-15 14:55:40.754 Z,37209,2023,9
42416,2022-07-01,2022-09-30,90,zip code,2,15704,False,Zip Code: 37127,NaN,Tennessee,...,NaN,0.371429,-0.146557,-0.391283,"Nashville, TN",34980,2025-12-15 14:55:40.754 Z,37127,2022,9
49276,2023-02-01,2023-04-30,90,zip code,2,15676,False,Zip Code: 37075,NaN,Tennessee,...,NaN,0.436090,0.080878,-0.395370,"Nashville, TN",34980,2025-12-15 14:55:40.754 Z,37075,2023,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9338495,2021-07-01,2021-09-30,90,zip code,2,15759,False,Zip Code: 37208,NaN,Tennessee,...,NaN,0.534884,-0.085369,0.116080,"Nashville, TN",34980,2025-12-15 14:55:40.754 Z,37208,2021,9
9339199,2022-08-01,2022-10-31,90,zip code,2,15772,False,Zip Code: 37221,NaN,Tennessee,...,NaN,0.453846,-0.060238,-0.293213,"Nashville, TN",34980,2025-12-15 14:55:40.754 Z,37221,2022,10
9345578,2023-01-01,2023-03-31,90,zip code,2,15769,False,Zip Code: 37218,NaN,Tennessee,...,NaN,0.362069,0.100164,-0.323645,"Nashville, TN",34980,2025-12-15 14:55:40.754 Z,37218,2023,3
9347046,2020-09-01,2020-11-30,90,zip code,2,15705,False,Zip Code: 37128,NaN,Tennessee,...,NaN,0.608315,0.019744,0.202447,"Nashville, TN",34980,2025-12-15 14:55:40.754 Z,37128,2020,11


In [24]:
redfin_zip.columns

Index(['PERIOD_BEGIN', 'PERIOD_END', 'PERIOD_DURATION', 'REGION_TYPE',
       'REGION_TYPE_ID', 'TABLE_ID', 'IS_SEASONALLY_ADJUSTED', 'REGION',
       'CITY', 'STATE', 'STATE_CODE', 'PROPERTY_TYPE', 'PROPERTY_TYPE_ID',
       'MEDIAN_SALE_PRICE', 'MEDIAN_SALE_PRICE_MOM', 'MEDIAN_SALE_PRICE_YOY',
       'MEDIAN_LIST_PRICE', 'MEDIAN_LIST_PRICE_MOM', 'MEDIAN_LIST_PRICE_YOY',
       'MEDIAN_PPSF', 'MEDIAN_PPSF_MOM', 'MEDIAN_PPSF_YOY', 'MEDIAN_LIST_PPSF',
       'MEDIAN_LIST_PPSF_MOM', 'MEDIAN_LIST_PPSF_YOY', 'HOMES_SOLD',
       'HOMES_SOLD_MOM', 'HOMES_SOLD_YOY', 'PENDING_SALES',
       'PENDING_SALES_MOM', 'PENDING_SALES_YOY', 'NEW_LISTINGS',
       'NEW_LISTINGS_MOM', 'NEW_LISTINGS_YOY', 'INVENTORY', 'INVENTORY_MOM',
       'INVENTORY_YOY', 'MONTHS_OF_SUPPLY', 'MONTHS_OF_SUPPLY_MOM',
       'MONTHS_OF_SUPPLY_YOY', 'MEDIAN_DOM', 'MEDIAN_DOM_MOM',
       'MEDIAN_DOM_YOY', 'AVG_SALE_TO_LIST', 'AVG_SALE_TO_LIST_MOM',
       'AVG_SALE_TO_LIST_YOY', 'SOLD_ABOVE_LIST', 'SOLD_ABOVE_LIST_MOM',
 

In [25]:
mortgage_rate=pd.read_csv(r'C:\Users\lukem\OneDrive\Desktop\NSS\Capstone\Capstone-Project\Data/mortgage_rate_30.csv')

In [26]:
mortgage_rate['observation_date'] = pd.to_datetime(mortgage_rate['observation_date'])

In [27]:
mortgage_rate['Year'] = mortgage_rate['observation_date'].dt.year
mortgage_rate['Month_Name'] = mortgage_rate['observation_date'].dt.month_name()

In [28]:
mortgage_rate = mortgage_rate[(mortgage_rate['Year'] >= 2020) & (mortgage_rate['Year'] <= 2023)]

In [29]:
mortgage_rate = mortgage_rate.reset_index(drop=True)

In [30]:
mortgage_rate.head()

,observation_date,MORTGAGE30US,Year,Month_Name
0,2020-01-02,3.72,2020,January
1,2020-01-09,3.64,2020,January
2,2020-01-16,3.65,2020,January
3,2020-01-23,3.60,2020,January
4,2020-01-30,3.51,2020,January


In [31]:
API_KEY = '5206a3fd5690e9a1ab1ce6fb7d2fbdbd52895891'

In [32]:
zip_codes = ['37201','37203','37204','37205','37206','37207',
             '37208','37209','37210','37211','37212','37214',
             '37215','37216','37217','37218','37220','37221',
             '37127','37128','37129','37130','37027','37064',
             '37067','37069','37066','37075','37076','37087',
             '37122']

In [33]:
variables = {
    'B01003_001E': 'Population',
    'B19013_001E': 'MedianIncome',
    'B25077_001E': 'MedianHomeValue',
    'B25064_001E': 'MedianRent',
    'B25002_001E': 'TotalHousingUnits',
    'B25003_002E': 'OccupiedUnits',
    'B25003_003E': 'VacantUnits',
    'B17001_001E': 'PovertyUniverse',
    'B17001_002E': 'BelowPoverty'}

In [34]:
years = ['2020', '2021', '2022', '2023']

In [35]:
def get_acs_data(year):
    base_url = f'https://api.census.gov/data/{year}/acs/acs5'
    all_data = []
    for zcta in zip_codes:
        response = requests.get(base_url, params={
            'get': ','.join(variables.keys()),
            'for': f'zip code tabulation area:{zcta}',
            'key': API_KEY})
        if response.status_code == 200 and len(response.json()) > 1:
            row = response.json()[1]
            record = {'ZCTA': zcta, 'ACS_Year': year}
            for i, var in enumerate(variables.keys()):
                record[variables[var]] = row[i]
            all_data.append(record)
        else:
            print(f'No data for ZCTA {zcta} in {year}')
    return pd.DataFrame(all_data)

In [ ]:
df_list = []
for year in years:
    print(f'Pulling ACS {year} data...')
    df_year = get_acs_data(year)
    df_list.append(df_year)
census_data = pd.concat(df_list, ignore_index=True)

Pulling ACS 2020 data...
Pulling ACS 2021 data...
Pulling ACS 2022 data...
Pulling ACS 2023 data...


In [ ]:
census_data

In [ ]:
census_data['ZCTA'] = census_data['ZCTA'].astype(int)

In [ ]:
numeric_cols = [
    'ACS_Year',
    'Population',
    'MedianIncome',
    'MedianHomeValue',
    'MedianRent',
    'TotalHousingUnits',
    'OccupiedUnits',
    'VacantUnits',
    'PovertyUniverse',
    'BelowPoverty']

In [ ]:
for col in numeric_cols:
    census_data[col] = pd.to_numeric(census_data[col], errors='coerce')

In [ ]:
census_data = census_data.replace(-666666666, pd.NA)

In [ ]:
census_data

In [ ]:
census_data['weighted_income'] = census_data['MedianIncome'] * census_data['Population']

In [ ]:
annual_income = (census_data.groupby('ACS_Year').agg(total_weighted_income=('weighted_income', 'sum'),total_population=('Population', 'sum'))
.reset_index())

In [ ]:
annual_income['Weighted_Avg_Median_Income'] = (annual_income['total_weighted_income'] / annual_income['total_population'])

In [ ]:
annual_income = annual_income.rename(columns={'ACS_Year': 'Year'})

In [ ]:
annual_income

* Total homes sales over the span of the dataset 2020-2023

In [ ]:
total_home_sales_2020 = redfin_metro.loc[redfin_metro['Year'] == 2020, 'Homes Sold'].sum()

In [ ]:
print(total_home_sales_2020)

In [ ]:
total_home_sales_2023 = redfin_metro.loc[redfin_metro['Year'] == 2023, 'Homes Sold'].sum()

In [ ]:
print(total_home_sales_2023)

* Average Mortagae rate for a 30 year fixed loan

In [ ]:
avg_mortgage_rate_2020 = (mortgage_rate.loc[mortgage_rate['Year'] == 2020, 'MORTGAGE30US'].mean().round(2))

In [ ]:
print(avg_mortgage_rate_2020)

In [ ]:
avg_mortgage_rate_2023 = (mortgage_rate.loc[mortgage_rate['Year'] == 2023, 'MORTGAGE30US'].mean().round(2))

In [ ]:
print(avg_mortgage_rate_2023)

1. How did median home sale prices in the Metro Nashville area change between 2020 and 2023, and did household income growth keep pace with these changes?

In [ ]:
metro_trends = annual_prices.merge(annual_income,on='Year',how='inner')

In [ ]:
metro_trends

In [ ]:
price_growth = ((metro_trends.loc[metro_trends['Year'] == 2023, 'Weighted_Avg_Sale_Price'].values[0] /metro_trends.loc[metro_trends['Year'] == 2020, 'Weighted_Avg_Sale_Price'].values[0] - 1)* 100)

In [ ]:
print(price_growth)

In [ ]:
income_growth = ((metro_trends.loc[metro_trends['Year'] == 2023, 'Weighted_Avg_Median_Income'].values[0] /metro_trends.loc[metro_trends['Year'] == 2020, 'Weighted_Avg_Median_Income'].values[0] - 1)* 100)

In [ ]:
print(income_growth)

In [ ]:
metro_trends['Price_to_Income_Ratio'] = ((metro_trends['Weighted_Avg_Sale_Price'] /metro_trends['Weighted_Avg_Median_Income']).round(2))

In [ ]:
metro_trends

In [ ]:
from matplotlib.ticker import FuncFormatter

In [ ]:
def dollar_format(x, pos):
    return f'${x:,.0f}'
formatter = FuncFormatter(dollar_format)

In [ ]:
plt.figure(figsize=(14, 6), facecolor='#F2F2F2')
plt.subplot(1, 2, 1)
plt.gca().set_facecolor('#F2F2F2')
plt.plot(
    metro_trends['Year'],
    metro_trends['Weighted_Avg_Sale_Price'],
    linewidth=3,
    color='#1F77B4',
    marker='o')
plt.title('Median Home Sale Price', fontweight='bold')
plt.xticks(metro_trends['Year'])
plt.ylim(0, metro_trends['Weighted_Avg_Sale_Price'].max() * 1.1)
plt.gca().yaxis.set_major_formatter(formatter)
plt.grid(alpha=0.25)
price_growth = (
    (metro_trends['Weighted_Avg_Sale_Price'].iloc[-1] -
     metro_trends['Weighted_Avg_Sale_Price'].iloc[0]) /
     metro_trends['Weighted_Avg_Sale_Price'].iloc[0]) * 100
plt.annotate(
    f"+{price_growth:.0f}%",
    xy=(metro_trends['Year'].iloc[-1],
        metro_trends['Weighted_Avg_Sale_Price'].iloc[-1]),
    xytext=(metro_trends['Year'].iloc[-2],
            metro_trends['Weighted_Avg_Sale_Price'].iloc[-1] * 0.75),
    arrowprops=dict(arrowstyle='->'),
    fontsize=14,
    fontweight='bold')
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

plt.subplot(1, 2, 2)
plt.gca().set_facecolor('#F2F2F2')
plt.plot(
    metro_trends['Year'],
    metro_trends['Weighted_Avg_Median_Income'],
    linewidth=3,
    color='#2CA02C',
    marker='o')
plt.title('Median Household Income', fontweight='bold')
plt.xticks(metro_trends['Year'])
plt.ylim(0, metro_trends['Weighted_Avg_Median_Income'].max() * 1.1)
plt.gca().yaxis.set_major_formatter(formatter)
plt.grid(alpha=0.25)
income_growth = (
    (metro_trends['Weighted_Avg_Median_Income'].iloc[-1] -
     metro_trends['Weighted_Avg_Median_Income'].iloc[0]) /
     metro_trends['Weighted_Avg_Median_Income'].iloc[0]) * 100
plt.annotate(
    f"+{income_growth:.0f}%",
    xy=(metro_trends['Year'].iloc[-1],
        metro_trends['Weighted_Avg_Median_Income'].iloc[-1]),
    xytext=(metro_trends['Year'].iloc[-2],
            metro_trends['Weighted_Avg_Median_Income'].iloc[-1] * 0.75),
    arrowprops=dict(arrowstyle='->'),
    fontsize=14,
    fontweight='bold')
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.tight_layout()
plt.savefig('slide_6.png', dpi=300, bbox_inches='tight',facecolor='#F2F2F2')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6), facecolor='#F2F2F2')
plt.gca().set_facecolor('#F2F2F2')
plt.plot(
    metro_trends['Year'],
    metro_trends['Price_to_Income_Ratio'],
    linewidth=3,
    color='#9467BD',
    marker='o')
plt.title('Home Price-to-Income Ratio', fontweight='bold')
plt.ylabel('Price-to-Income Ratio')
plt.xticks(metro_trends['Year'])
y_max = metro_trends['Price_to_Income_Ratio'].max()
plt.ylim(0, y_max * 1.05)
plt.gca().yaxis.set_major_formatter(lambda x, pos: f'{x:.0f}%')
plt.grid(alpha=0.25)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.tight_layout()
plt.savefig('slide_7.png', dpi=300, bbox_inches='tight',facecolor='#F2F2F2')
plt.show()

2. How did housing supply conditions measured by inventory levels and homes sold change between 2020 and 2023, and how did these trends relate to rising home prices in Metro Nashville?

In [ ]:
yearly_supply = (redfin_metro.groupby('Year').agg(
        Inventory_Avg=('Inventory', 'mean'),
        Homes_Sold_Total=('Homes Sold', 'sum'),
        Homes_Sold_Avg_Month=('Homes Sold', 'mean'),
        Days_on_Market_Median=('Days on Market', 'median'),).reset_index())

In [ ]:
yearly_supply = yearly_supply.merge(annual_prices[['Year', 'Weighted_Avg_Sale_Price']],on='Year',how='left')

In [ ]:
yearly_supply

In [ ]:
affordability = yearly_supply.merge(annual_income, on='Year', how='inner')

In [ ]:
affordability['Price_to_Income_Ratio'] = (affordability['Weighted_Avg_Sale_Price'] / affordability['Weighted_Avg_Median_Income'])

In [ ]:
affordability

In [ ]:
dollar = FuncFormatter(lambda x, pos: f'${x:,.0f}')
comma = FuncFormatter(lambda x, pos: f'{x:,.0f}')

In [ ]:
plt.figure(facecolor='#F2F2F2', figsize=(10, 6))
plt.subplot(2, 1, 1)
plt.gca().set_facecolor('#F2F2F2')
plt.plot(
    affordability['Year'],
    affordability['Inventory_Avg'],
    linewidth=3,
    color='#1F77B4',
    marker='o')
plt.title('Inventory Levels (Average by Year)', fontweight='bold')
plt.ylabel('Inventory (Homes)')
plt.ylim(bottom=0)
plt.gca().yaxis.set_major_formatter(comma)
plt.xticks(affordability['Year'])
plt.grid(alpha=0.25)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)

plt.subplot(2, 1, 2)
plt.gca().set_facecolor('#F2F2F2')
plt.plot(
    affordability['Year'],
    affordability['Homes_Sold_Total'],
    linewidth=3,
    color='#FF7F0E',
    marker='o')
plt.title('Homes Sold (Total by Year)', fontweight='bold')
plt.ylabel('Homes Sold')
plt.ylim(bottom=0)
plt.gca().yaxis.set_major_formatter(comma)
plt.xticks(affordability['Year'])
plt.grid(alpha=0.25)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.tight_layout()
plt.savefig('slide_8.png', dpi=300, bbox_inches='tight',facecolor='#F2F2F2')
plt.show()

3. How did changes in 30-year fixed mortgage interest rates between 2020 and 2023 affect homeownership affordability for a median-income household in Metro Nashville, and how did the estimated affordability of a median-priced home change during this period?

In [ ]:
mortgage_annual = (mortgage_rate.groupby('Year')['MORTGAGE30US'].mean().reset_index(name='Avg_30yr_Rate'))

In [ ]:
mortgage_annual

In [ ]:
mortgage_afford = affordability.merge(mortgage_annual, on='Year', how='inner')

In [ ]:
mortgage_afford

In [ ]:
def monthly_payment(price, rate, down_payment=0.20, years=30):
    loan_amount = price * (1 - down_payment)
    monthly_rate = rate / 100 / 12
    n_payments = years * 12
    return loan_amount * (monthly_rate * (1 + monthly_rate)**n_payments) / ((1 + monthly_rate)**n_payments - 1)
mortgage_afford['Monthly_Payment'] = mortgage_afford.apply(
    lambda row: monthly_payment(row['Weighted_Avg_Sale_Price'],row['Avg_30yr_Rate']),axis=1)

In [ ]:
mortgage_afford['Monthly_Income'] = mortgage_afford['Weighted_Avg_Median_Income'] / 12
mortgage_afford['Income_Share_for_Mortgage'] = (mortgage_afford['Monthly_Payment'] /mortgage_afford['Monthly_Income'])

In [ ]:
plt.figure(figsize=(10, 6), facecolor='#F2F2F2')
plt.gca().set_facecolor('#F2F2F2')
plt.plot(
    mortgage_afford['Year'],
    mortgage_afford['Avg_30yr_Rate'],
    linewidth=3,
    color='#D62728',
    marker='o')
plt.title('Average 30-Year Mortgage Rate', fontweight='bold')
plt.ylabel('Interest Rate')
plt.xticks(mortgage_afford['Year'])
plt.ylim(bottom=0)
plt.gca().yaxis.set_major_formatter(lambda x, pos: f'{x:.1f}%')
plt.grid(alpha=0.25)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.tight_layout()
plt.savefig('slide_9.png', dpi=300, bbox_inches='tight',facecolor='#F2F2F2')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6), facecolor='#F2F2F2')
plt.gca().set_facecolor('#F2F2F2')
plt.plot(
    mortgage_afford['Year'],
    mortgage_afford['Income_Share_for_Mortgage'] * 100,
    linewidth=3,
    color='#8C564B', 
    marker='o',
    label='Income Required for Mortgage')
plt.axhline(
    30,
    linestyle='--',
    linewidth=2,
    color='#7F7F7F',
    alpha=0.8,
    label='30% Affordability Threshold')
plt.title('Share of Median Income Required for Mortgage Payments', fontweight='bold')
plt.ylabel('Percent of Monthly Income')
plt.xticks(mortgage_afford['Year'])
plt.yticks(np.arange(0, 41, 5))
plt.ylim(bottom=0)
plt.gca().yaxis.set_major_formatter(lambda x, pos: f'{x:.0f}%')
plt.legend(frameon=False)
plt.grid(alpha=0.25)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.tight_layout()
plt.savefig('slide_10.png', dpi=300, bbox_inches='tight',facecolor='#F2F2F2')
plt.show()

4. Were changes in housing affordability evenly distributed across Metro Nashville between 2020 and 2023, or did certain ZIP codes experience more rapid changes in affordability pressure, as measured by changes in home sale prices relative to household income?

In [ ]:
zip_prices = (redfin_zip.groupby(['ZIP','Year'])['MEDIAN_SALE_PRICE'].median().reset_index())

In [ ]:
zip_prices

In [ ]:
zip_prices_wide = zip_prices.pivot(index='ZIP',columns='Year',values='MEDIAN_SALE_PRICE').reset_index()

In [ ]:
zip_prices_wide = zip_prices_wide[['ZIP', 2020, 2023]]

In [ ]:
zip_prices_wide.columns = ['ZIP','Price_2020','Price_2023']

In [ ]:
zip_prices_wide.head()

In [ ]:
zip_income = (census_data.groupby(['ZCTA','ACS_Year'])['MedianIncome'].median().reset_index().pivot(index='ZCTA', columns='ACS_Year', values='MedianIncome').reset_index())

In [ ]:
zip_income.head()

In [ ]:
zip_income_20_23 = zip_income[['ZCTA', 2020, 2023]].copy()

In [ ]:
zip_income_20_23.columns = ['ZIP', 'Income_2020', 'Income_2023']

In [ ]:
zip_income_20_23.head()

In [ ]:
zip_prices_wide['ZIP'] = zip_prices_wide['ZIP'].astype(str)
zip_income_20_23['ZIP'] = zip_income_20_23['ZIP'].astype(str)

In [ ]:
zip_afford = zip_prices_wide.merge(zip_income_20_23, on='ZIP', how='inner')

In [ ]:
zip_afford.head()

In [ ]:
zip_afford['Ratio_2020'] = zip_afford['Price_2020'] / zip_afford['Income_2020']
zip_afford['Ratio_2023'] = zip_afford['Price_2023'] / zip_afford['Income_2023']

In [ ]:
zip_afford['Affordability_Change_%'] = ((zip_afford['Ratio_2023'] / zip_afford['Ratio_2020'] - 1) * 100)

In [ ]:
zip_afford.head()

In [ ]:
top_changes = zip_afford.sort_values('Affordability_Change_%', ascending=False).head(10)

In [ ]:
plt.figure(figsize=(10, 6), facecolor='#F2F2F2')
plt.gca().set_facecolor('#F2F2F2')
plt.barh(
    top_changes['ZIP'].astype(str),
    top_changes['Affordability_Change_%'],
    color='#17BECF')
plt.title(
    'ZIP Codes with Largest Increase in Affordability Pressure',
    fontweight='bold')
plt.xlabel('Increase in Price-to-Income Ratio')
plt.gca().invert_yaxis()
plt.gca().xaxis.set_major_formatter(lambda x, pos: f'{x:.0f}%')
plt.grid(axis='x', alpha=0.25)
plt.gca().spines['top'].set_visible(False)
plt.gca().spines['right'].set_visible(False)
plt.tight_layout()
plt.savefig('slide_11.png', dpi=300, bbox_inches='tight',facecolor='#F2F2F2')
plt.show()

In [ ]:
redfin_metro.to_csv('redfin_metro.csv', index=False)

In [ ]:
redfin_zip.to_csv('redfin_zip.csv', index=False)

In [ ]:
mortgage_rate.to_csv('mortgage_rate.csv', index=False)

In [ ]:
census_data.to_csv('census_data.csv', index=False)